In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.testing import save_labeled_video

%load_ext autoreload
%autoreload 2

# Analyze results

## Read results DataFrame

In [3]:
results_folder = './checkpoints/checkpoints_360_640_mse/checkpoint_0010_results/'
df_out = pd.read_csv(os.path.join(results_folder, 'val_output.csv'))

In [5]:
df_out

,dataset_id,frame_num,min_values,max_values,x_true,y_true,x_pred,y_pred
0,0,14614,4.896332e-07,0.153187,0.187500,0.600000,0.181250,0.616667
1,0,14615,2.152525e-07,0.001734,0.167187,0.602778,0.998437,0.997222
2,0,14616,1.969889e-06,0.009495,0.151562,0.602778,0.154688,0.602778
3,0,14617,3.162235e-08,0.006769,0.131250,0.611111,0.137500,0.605556
4,0,14618,4.621421e-08,0.007118,0.112500,0.616667,0.112500,0.613889
...,...,...,...,...,...,...,...,...
2842,1,5030,1.065431e-06,0.935209,0.525000,0.738889,0.525000,0.738889
2843,1,5031,8.761145e-07,0.967830,0.529687,0.747222,0.528125,0.747222
2844,1,5032,6.991232e-07,0.953655,0.532813,0.752778,0.532813,0.752778
2845,1,5033,6.252794e-07,0.939276,0.000000,0.000000,0.537500,0.752778


## Save output video with heatmap

In [8]:
save_labeled_video(filename_src='../datasets/dataset_finales_2020_en/video.mp4',
                   filename_dst='finales_val_annotated_gt_heatmap.mp4',
                   position_df=df_out.loc[df_out['dataset_id']==0],
                   show_ground_truth=True,
                   heatmaps_folder='checkpoints/checkpoints_360_640_mse/checkpoint_0010_results/heatmaps')

writing frame 20 of 20                                                          
Finished frames.
done.


In [10]:
save_labeled_video(filename_src='../datasets/dataset_lluis/game5/video.mp4',
                   filename_dst='game5_annotated_gt_heatmap.mp4',
                   position_df=df_out.loc[df_out['dataset_id']==1],
                   show_ground_truth=True,
                   heatmaps_folder='checkpoints/checkpoints_360_640_mse/checkpoint_0010_results/heatmaps')

writing frame 4229 of 4229                                                      
Finished frames.
done.
